In [1]:
import sklearn as sklearn
import scipy.stats as stats
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier

from utils.randomized_search import logUnifD, rs_results
from utils.base_set import X_train, y_train, seed

### KNN 

In [2]:
model = KNeighborsClassifier()
K = X_train.shape[0]
k = 5
cv = StratifiedKFold(n_splits=k, shuffle=True, random_state=seed)
params = {
    'n_neighbors': logUnifD(10, K/2),
    'weights': ['uniform', 'distance'],
    #'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'p': stats.randint(1, 4), # minkowski metric
}
n_iter = 100
classifier_KNN = RandomizedSearchCV(estimator=model, 
                                param_distributions=params,
                                cv=cv,
                                n_iter=n_iter,
                                scoring="roc_auc", 
                                random_state=seed)
classifier_KNN.fit(X_train, y_train)
rs_results(classifier_KNN.cv_results_, params)

,param_n_neighbors,param_weights,param_p,mean_test_score,rank_test_score
6,15,distance,1,0.839120,1
19,11,distance,1,0.838640,2
0,14,distance,1,0.837595,3
64,13,distance,1,0.836097,4
97,26,distance,1,0.828764,5
...,...,...,...,...,...
26,200,distance,3,0.747929,96
93,165,uniform,3,0.742176,97
80,171,uniform,3,0.741821,98
32,198,uniform,3,0.740723,99
